<a href="https://colab.research.google.com/github/thursy/Apache-Beam-Notebook/blob/main/2_Beam_Streaming_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install google-cloud-pubsub
!pip3 install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Filtering example 

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window

In [ ]:
project = 'PROJECT_ID'
input_subscription = 'projects/PROJECT_ID/subscriptions/SUBS_NAME'  #to get the data
comedy_movies = 'projects/PROJECT_ID/topics/TOPIC_NAME' #topic to publish the result of calculation

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= 'PATH_TO_SERVICE_ACCOUNT.json'

In [ ]:
options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(options=options)

In [ ]:
pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Filter movies with comedy genre' >> beam.Filter(lambda row: ("Comedy" in row[2]))
    | 'Converting to byte String' >> beam.Map(lambda row: (''.join(row).encode('utf-8')) )
    | 'Publish to output topic' >> beam.io.WriteToPubSub(comedy_movies)
)
result = p.run()
result.wait_until_finish()

## TUMBLING or FIXED

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
import time
from apache_beam import window
from apache_beam.transforms.combiners import Count
from apache_beam.transforms.trigger import AfterWatermark,AfterProcessingTime,AccumulationMode,AfterCount

In [ ]:
project = 'PROJECT_ID'
input_subscription = 'projects/PROJECT_ID/subscriptions/SUBS_NAME'  #to get the data
comedy_movies = 'projects/PROJECT_ID/topics/TOPIC_NAME' #topic to publish the result of calculation

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= 'PATH_TO_SERVICE_ACCOUNT.json'

In [ ]:
options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(options=options)

In [ ]:
pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'With Custom TimeStamp' >> beam.Map(lambda row: beam.window.TimestampedValue(row,time.time()))
    | 'Form Key Value Pair' >> beam.Map(lambda row: (row[1],int(row[2])))
    | 'Window' >> beam.WindowInto(window.FixedWindows(10))
    #| 'Window' >> beam.WindowInto(window.SlidingWindows(4,2)) ##uncomment for sliding window
    #| 'Create Session Window' >> beam.WindowInto(window.Sessions(25)) ##uncomment for session window
    #| 'Window' >> beam.WindowInto(window.FixedWindows(10), allowed_lateness=Duration(seconds=60)) ##uncomment for fixed window with allowed lateness 
    #| 'Window' >> beam.WindowInto(window.FixedWindows(10),trigger=AfterProcessingTime(5),accumulation_mode=AccumulationMode.DISCARDNG) ##uncomment for fixed window with specified trigger
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda row: (''.join(row).encode('utf-8')) )
    | 'Publish to output topic' >> beam.io.WriteToPubSub(comedy_movies)
)
result = p.run()
result.wait_until_finish()